In [175]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels as sm
import seaborn as sns
from matplotlib import font_manager, rc, rcParams

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

%matplotlib inline

In [176]:
df = pd.read_csv('input/training_new.csv', names=['발신일자', '발송우체국', '도착구', '수량'])
df['발신일자'] = pd.to_datetime(df['발신일자'], format='%Y%m%d')

In [177]:
df1 = df.copy()

In [178]:
df1.set_index('발신일자', inplace=True)
df1.head()

,발송우체국,도착구,수량
발신일자,,,
2017-12-01,가평우체국,1,11
2017-12-01,가평우체국,24,10
2017-12-01,가평우체국,17,8
2017-12-01,가평우체국,9,5
2017-12-01,가평우체국,14,11


In [179]:
df1.shape

(1338140, 3)

In [191]:
train = df1.groupby(['도착구', pd.Grouper(freq='D')])[['수량']].agg(np.sum)
print(train.shape)
train.head()

(8604, 1)


수량
도착구 발신일자            
0   2017-12-01  5829
    2017-12-02   514
    2017-12-03   186
    2017-12-04  8525
    2017-12-05  6743

In [192]:
dates = ['2017-12-24', '2017-12-25', '2017-12-26', '2018-08-14', '2018-08-15', '2018-08-16', '2018-09-21', 
         '2018-09-22', '2018-09-23', '2018-09-24', '2018-09-25', '2018-09-26', '2018-09-27', '2018-10-02', 
         '2018-10-03', '2018-10-04', '2018-10-08', '2018-10-09', '2018-10-10']

In [193]:
temp = pd.DataFrame()
temp['도착구'] = np.re


peat(list(range(0, 25)), len(dates))

In [194]:
temp['발신일자'] = dates * 25
temp['발신일자'] = pd.to_datetime(temp['발신일자'])
temp['수량'] = None
# temp['발송우체국'] = None

In [195]:
temp.set_index('발신일자', inplace=True)

In [196]:
temp.head()

,도착구,수량
발신일자,,
2017-12-24,0,None
2017-12-25,0,None
2017-12-26,0,None
2018-08-14,0,None
2018-08-15,0,None


In [197]:
train = train.unstack(0).stack().reset_index('도착구')

In [198]:
train = pd.concat([train, temp])

In [199]:
train['요일'] = train.index.dayofweek
train['월'] = train.index.month
train['일'] = train.index.day
train.head()

,도착구,수량,요일,월,일
발신일자,,,,,
2017-12-01,0,5829.0,4,12,1
2017-12-01,1,7455.0,4,12,1
2017-12-01,2,10086.0,4,12,1
2017-12-01,3,5929.0,4,12,1
2017-12-01,4,14913.0,4,12,1


In [200]:
train = pd.get_dummies(train, columns=['요일'])
train.head()

,도착구,수량,월,일,요일_0,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6
발신일자,,,,,,,,,,,
2017-12-01,0,5829.0,12,1,0,0,0,0,1,0,0
2017-12-01,1,7455.0,12,1,0,0,0,0,1,0,0
2017-12-01,2,10086.0,12,1,0,0,0,0,1,0,0
2017-12-01,3,5929.0,12,1,0,0,0,0,1,0,0
2017-12-01,4,14913.0,12,1,0,0,0,0,1,0,0


In [201]:
test = train[train['수량'].isnull()]
test.head()

,도착구,수량,월,일,요일_0,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6
발신일자,,,,,,,,,,,
2017-12-24,0,NaN,12,24,0,0,0,0,0,0,1
2017-12-25,0,NaN,12,25,1,0,0,0,0,0,0
2017-12-26,0,NaN,12,26,0,1,0,0,0,0,0
2018-08-14,0,NaN,8,14,0,1,0,0,0,0,0
2018-08-15,0,NaN,8,15,0,0,1,0,0,0,0


In [202]:
train = train[(~train['수량'].isnull())]

In [203]:
train.shape

(8604, 11)

In [204]:
submission = test.copy()

In [205]:
# submission.drop(['월', '일', '요일_0', '요일_1', '요일_2', '요일_3', '요일_4', '요일_5', '요일_6'], axis=1, inplace=True)
submission.drop(test.columns[2:], axis=1, inplace=True)
submission.head()

,도착구,수량
발신일자,,
2017-12-24,0,NaN
2017-12-25,0,NaN
2017-12-26,0,NaN
2018-08-14,0,NaN
2018-08-15,0,NaN


In [207]:
train.to_csv('train.csv')
submission.to_csv('input/submission.csv')

In [209]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
import xgboost as xgb

In [216]:
for i in train['도착구'].unique():
    reg1 = xgb.XGBRegressor(random_state=42, max_depth=5, n_estimators=100).fit(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량'])
    print('xgb score %f' % reg1.score(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량']))
    pred1 = reg1.predict(test[test['도착구'] == i].drop(['수량'], axis=1))
    
    reg2 = LinearRegression().fit(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량'])
    pred2 = reg2.predict(test[test['도착구'] == i].drop(['수량'], axis=1))
    print('linear score %f' % reg2.score(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량']))
    
    reg3 = SVR().fit(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량'])
    pred3 = reg3.predict(test[test['도착구'] == i].drop(['수량'], axis=1))
    print('svr score %f' % reg3.score(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량']))
    
    reg4 = RandomForestRegressor(random_state=42).fit(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량'])
    pred4 = reg4.predict(test[test['도착구'] == i].drop(['수량'], axis=1))
    print('rf score %f' % reg4.score(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량']))
    
    reg5 = Ridge().fit(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량'])
    pred5 = reg5.predict(test[test['도착구'] == i].drop(['수량'], axis=1))
    print('rf score %f' % reg5.score(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량']))
    
    
    submission['수량'][submission['도착구'] == i] = ((pred1 + pred2 + pred3 + pred4 + pred5) / 5).tolist()
    

xgb score 0.987713
linear score 0.823032
svr score -0.101764
rf score 0.969973
rf score 0.822710
xgb score 0.987983
linear score 0.848640
svr score -0.182687
rf score 0.965177
rf score 0.848308


/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


xgb score 0.985907
linear score 0.841181
svr score -0.080605
rf score 0.967775
rf score 0.840852
xgb score 0.985387
linear score 0.798744
svr score -0.069145
rf score 0.966527
rf score 0.798433


/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


xgb score 0.982397
linear score 0.818972
svr score -0.082079
rf score 0.961950
rf score 0.818654
xgb score 0.983938
linear score 0.797351
svr score -0.070388
rf score 0.971408
rf score 0.797042


...

/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


xgb score 0.986098
linear score 0.825844
svr score -0.059426
rf score 0.967503
rf score 0.825523
xgb score 0.985041
linear score 0.751966
svr score -0.058598
rf score 0.964775
rf score 0.751672


/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


xgb score 0.987033
linear score 0.787548
svr score -0.066222
rf score 0.965894
rf score 0.787241
xgb score 0.984619
linear score 0.751132
svr score -0.041140
rf score 0.965091
rf score 0.750840
xgb score 0.985570
linear score 0.778172
svr score -0.045082
rf score 0.966952
rf score 0.777868


/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [217]:
# for i in train['도착구'].unique():
#     reg = RandomForestRegressor(random_state=42).fit(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량'])
#     print('train score %f' % reg.score(train[train['도착구'] == i].drop(['수량'], axis=1), train[train['도착구'] == i]['수량']))
#     submission['수량'][submission['도착구'] == i] = reg.predict(test[test['도착구'] == i].drop(['수량'], axis=1)).tolist()

In [218]:
submission.shape

(475, 2)

In [219]:
submission['수량'] = submission['수량'].astype('int')

In [220]:
submission.to_csv('submission0119-ensemble.csv', header=False)

[246.5,
 8515.0,
 5538.0,
 4675.6,
 3784.0,
 4939.7,
 7633.3,
 1112.6,
 927.5,
 13413.3,
 8790.1,
 7478.7,
 7310.2,
 5770.7,
 5739.8,
 5883.3,
 8990.5,
 5770.7,
 5776.9]

In [84]:
submission['수량'][submission['도착구'] == 1] = reg.predict(test[test['도착구'] == 1].drop(['수량'], axis=1)).tolist()

/Users/unerue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [85]:
submission[submission['도착구'] == 1]

,도착구,수량
발신일자,,
2017-12-24,1,246.5
2017-12-25,1,8515.0
2017-12-26,1,5538.0
2018-08-14,1,4675.6
2018-08-15,1,3784.0
2018-08-16,1,4939.7
2018-09-21,1,7633.3
2018-09-22,1,1112.6
2018-09-23,1,927.5


In [15]:
df1 = pd.concat([df1, temp], sort=True)

In [16]:
df1['요일'] = df1.index.dayofweek
df1['월'] = df1.index.month
df1['일'] = df1.index.day
df1.head()

,도착구,발송우체국,수량,요일,월,일
발신일자,,,,,,
2017-12-01,1,가평우체국,11,4,12,1
2017-12-01,24,가평우체국,10,4,12,1
2017-12-01,17,가평우체국,8,4,12,1
2017-12-01,9,가평우체국,5,4,12,1
2017-12-01,14,가평우체국,11,4,12,1


In [27]:
seoul = pd.read_excel('input/seoul.xlsx', sheet_name=0, header=0)
seoul.head()

,구명,번호
0,종로구,0
1,중구,1
2,용산구,2
3,성동구,3
4,광진구,4


In [28]:
names = {}
for i, j in seoul.iterrows():
    names[i] = j.values[0]

In [29]:
names

{0: '종로구',
 1: '중구',
 2: '용산구',
 3: '성동구',
 4: '광진구',
 5: '동대문구',
 6: '중랑구',
 7: '성북구',
 8: '강북구',
 9: '도봉구',
 10: '노원구',
 11: '은평구',
 12: '서대문구',
 13: '마포구',
 14: '양천구',
 15: '강서구',
 16: '구로구',
 17: '금천구',
 18: '영등포구',
 19: '동작구',
 20: '관악구',
 21: '서초구',
 22: '강남구',
 23: '송파구',
 24: '강동구'}

In [20]:
df1['도착구'].replace(names, inplace=True)

In [21]:
weather = pd.read_csv('input/weather.csv', encoding='cp949')
weather['일시'] = pd.to_datetime(weather['일시'])
weather.head()

,지점,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%)
0,108,2017-12-01,-2.2,-7.6,3.1,0.0,1.6,49.4
1,108,2017-12-02,1.9,-4.6,8.1,NaN,1.7,59.9
2,108,2017-12-03,4.9,3.1,8.1,4.0,1.5,83.1
3,108,2017-12-04,-1.2,-6.1,4.5,0.1,3.4,48.1
4,108,2017-12-05,-4.2,-8.2,-0.4,0.1,NaN,40.1


In [22]:
merge1 = pd.merge(df1, weather, left_on=df1.index, right_on='일시', how='left')
print(merge1.shape)
merge1.tail()

(1338615, 14)


,도착구,발송우체국,수량,요일,월,일,지점,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균 풍속(m/s),평균 상대습도(%)
1338610,강동구,None,None,2,10,3,108,2018-10-03,17.3,11.2,24.2,NaN,1.3,58.0
1338611,강동구,None,None,3,10,4,108,2018-10-04,19.3,13.0,25.6,NaN,1.3,56.5
1338612,강동구,None,None,0,10,8,108,2018-10-08,14.8,9.2,20.9,NaN,1.1,47.0
1338613,강동구,None,None,1,10,9,108,2018-10-09,15.3,10.7,19.8,0.0,1.1,52.5
1338614,강동구,None,None,2,10,10,108,2018-10-10,12.0,7.9,16.7,7.0,2.5,61.0


In [23]:
df5 = pd.read_csv('input/65세이상인구.csv')
df5.head()

,지역,전체인구,전체남자,전체여자,65세이상합계,65세이상남자,65세이상여자,65세이상내국인합계,65세이상내국인남자,65세이상내국인여자,65세이상외국인합계,65세이상외국인남자,65세이상외국인여자
0,종로구,163086,79301,83785,26622,11706,14916,26455,11612,14843,167,94,73
1,중구,135258,66554,68704,21902,9593,12309,21651,9460,12191,251,133,118
2,용산구,245087,119830,125257,37443,15785,21658,37015,15521,21494,428,264,164
3,성동구,317197,155652,161545,42581,18552,24029,42438,18467,23971,143,85,58
4,광진구,369999,179120,190879,45202,20146,25056,45016,20057,24959,186,89,97


In [24]:
df5 = df5.drop(['65세이상내국인합계', '65세이상내국인남자', '65세이상내국인여자', '65세이상외국인합계', '65세이상외국인남자', '65세이상외국인여자'], axis=1)
df5.head()

,지역,전체인구,전체남자,전체여자,65세이상합계,65세이상남자,65세이상여자
0,종로구,163086,79301,83785,26622,11706,14916
1,중구,135258,66554,68704,21902,9593,12309
2,용산구,245087,119830,125257,37443,15785,21658
3,성동구,317197,155652,161545,42581,18552,24029
4,광진구,369999,179120,190879,45202,20146,25056


In [25]:
merge2 = pd.merge(merge1, df5, left_on='도착구', right_on='지역', how='left')
print(merge2.shape)
merge2.head()

(1338615, 21)


,도착구,발송우체국,수량,요일,월,일,지점,일시,평균기온(°C),최저기온(°C),...,일강수량(mm),평균 풍속(m/s),평균 상대습도(%),지역,전체인구,전체남자,전체여자,65세이상합계,65세이상남자,65세이상여자
0,중구,가평우체국,11,4,12,1,108,2017-12-01,-2.2,-7.6,...,0.0,1.6,49.4,중구,135258,66554,68704,21902,9593,12309
1,강동구,가평우체국,10,4,12,1,108,2017-12-01,-2.2,-7.6,...,0.0,1.6,49.4,강동구,434992,215558,219434,58280,26608,31672
2,금천구,가평우체국,8,4,12,1,108,2017-12-01,-2.2,-7.6,...,0.0,1.6,49.4,금천구,252359,129482,122877,35299,16098,19201
3,도봉구,가평우체국,5,4,12,1,108,2017-12-01,-2.2,-7.6,...,0.0,1.6,49.4,도봉구,342990,167726,175264,55499,24634,30865
4,양천구,가평우체국,11,4,12,1,108,2017-12-01,-2.2,-7.6,...,0.0,1.6,49.4,양천구,469945,231360,238585,57442,25612,31830


In [26]:
# df6 = pd.read_csv('input/서울시인구추이통계.csv')
# df6.head()

In [27]:
df7 = pd.read_csv('input/종사자수.csv')
df7.head()

,구명,사업체수,총종사자수,총여성종사자,총남성종사자,자영업소계,자영업여성종사자,자영업남성종사자,상용근로자소계,상용여성종사자,상용남성종사자,임시직소계,임시직여성종사자,임시직남성종사자,기타소계,기타여성종사자,기타남성종사자
0,종로구,40664,268309,113975,154334,37659,15096,22563,196356,80945,115411,26582,13440,13142,7712,4494,3218
1,중구,63640,402892,181135,221757,63625,29100,34525,292930,123843,169087,32899,19714,13185,13438,8478,4960
2,용산구,20994,129033,56774,72259,17899,7538,10361,92005,38928,53077,15636,8054,7582,3493,2254,1239
3,성동구,26581,165047,68123,96924,23449,8920,14529,119154,47658,71496,17286,8837,8449,5158,2708,2450
4,광진구,24895,126937,55537,71400,24811,10437,14374,69697,30897,38800,25736,10586,15150,6693,3617,3076


In [28]:
merge3 = pd.merge(merge2, df7, left_on='도착구', right_on='구명', how='left')
print(merge3.shape)
merge3.head()

(1338615, 38)


,도착구,발송우체국,수량,요일,월,일,지점,일시,평균기온(°C),최저기온(°C),...,자영업남성종사자,상용근로자소계,상용여성종사자,상용남성종사자,임시직소계,임시직여성종사자,임시직남성종사자,기타소계,기타여성종사자,기타남성종사자
0,중구,가평우체국,11,4,12,1,108,2017-12-01,-2.2,-7.6,...,34525,292930,123843,169087,32899,19714,13185,13438,8478,4960
1,강동구,가평우체국,10,4,12,1,108,2017-12-01,-2.2,-7.6,...,18119,81381,37955,43426,21632,9536,12096,7596,5443,2153
2,금천구,가평우체국,8,4,12,1,108,2017-12-01,-2.2,-7.6,...,14680,170883,60908,109975,25442,11111,14331,5816,3163,2653
3,도봉구,가평우체국,5,4,12,1,108,2017-12-01,-2.2,-7.6,...,12060,38872,18860,20012,8766,5216,3550,2749,1786,963
4,양천구,가평우체국,11,4,12,1,108,2017-12-01,-2.2,-7.6,...,15026,66669,31125,35544,17588,8234,9354,7204,4055,3149


In [29]:
df8 = pd.read_csv('input/자치구인구.csv')
df8.head()

,기준일ID,시군구코드,시군구명,총생활인구수,내국인생활인구수,장기체류외국인인구수,단기체류외국인인구수,일최대인구수,일최소인구수,주간인구수(09~18),야간인구수(19~08),일최대이동인구수,서울외유입인구수,동일자치구행정동간이동인구수,자치구간이동인구수
0,20181214,11000,서울시,1.170342e+07,1.116966e+07,298723.7673,235035.1864,1.207861e+07,1.129600e+07,1.200531e+07,1.148778e+07,6.863482e+06,1.667482e+06,2.293202e+06,2.902798e+06
1,20181214,11110,종로구,3.697313e+05,3.401286e+05,14073.5021,15529.2542,5.044143e+05,2.484306e+05,4.721975e+05,2.965412e+05,3.363851e+05,8.831549e+04,3.929528e+04,2.087744e+05
2,20181214,11140,중구,4.275897e+05,3.331446e+05,17462.2974,76982.7613,5.922065e+05,2.824762e+05,5.576280e+05,3.347052e+05,3.954162e+05,1.074346e+05,3.366226e+04,2.543193e+05
3,20181214,11170,용산구,3.323658e+05,3.061771e+05,15889.5748,10299.1009,3.640563e+05,2.873641e+05,3.608672e+05,3.120076e+05,2.139593e+05,5.892514e+04,4.607716e+04,1.089570e+05
4,20181214,11200,성동구,3.800671e+05,3.646009e+05,13040.0285,2426.1347,4.008339e+05,3.602980e+05,3.930817e+05,3.707710e+05,2.310036e+05,4.420727e+04,7.221674e+04,1.145796e+05


In [30]:
df8 = df8.groupby(by='시군구명').agg(np.sum)
df8.head()

,기준일ID,시군구코드,총생활인구수,내국인생활인구수,장기체류외국인인구수,단기체류외국인인구수,일최대인구수,일최소인구수,주간인구수(09~18),야간인구수(19~08),일최대이동인구수,서울외유입인구수,동일자치구행정동간이동인구수,자치구간이동인구수
시군구명,,,,,,,,,,,,,,
강남구,5125922139,2966720,2.176257e+08,2.071318e+08,5.343041e+06,5.150906e+06,2.667988e+08,1.737141e+08,2.564919e+08,1.896922e+08,1.591061e+08,4.928203e+07,3.210793e+07,7.771616e+07
강동구,5125922139,2981960,1.256863e+08,1.227517e+08,2.633287e+06,3.013097e+05,1.348318e+08,1.152319e+08,1.178638e+08,1.311682e+08,5.808194e+07,1.211943e+07,3.380772e+07,1.215478e+07
강북구,5125922139,2871470,7.945831e+07,7.793989e+07,1.299902e+06,2.185265e+05,8.788346e+07,7.076052e+07,7.263740e+07,8.426276e+07,3.516245e+07,3.446473e+06,1.953363e+07,1.218235e+07
강서구,5125922139,2921000,1.440971e+08,1.392547e+08,2.531703e+06,2.310711e+06,1.534460e+08,1.346123e+08,1.369906e+08,1.490520e+08,6.780220e+07,1.608484e+07,3.493689e+07,1.678047e+07
관악구,5125922139,2951480,1.335393e+08,1.274264e+08,5.578992e+06,5.339037e+05,1.495449e+08,1.163722e+08,1.201706e+08,1.429741e+08,5.823499e+07,9.116628e+06,3.258344e+07,1.653492e+07


In [31]:
merge4 = pd.merge(merge3, df8, left_on='도착구', right_on=, how='left')
print(merge4.shape)
merge4.head()

(1338615, 52)


,도착구,발송우체국,수량,요일,월,일,지점,일시,평균기온(°C),최저기온(°C),...,장기체류외국인인구수,단기체류외국인인구수,일최대인구수,일최소인구수,주간인구수(09~18),야간인구수(19~08),일최대이동인구수,서울외유입인구수,동일자치구행정동간이동인구수,자치구간이동인구수
0,중구,가평우체국,11,4,12,1,108,2017-12-01,-2.2,-7.6,...,6.159749e+06,1.392533e+07,1.335592e+08,6.442079e+07,1.241011e+08,7.566488e+07,8.679954e+07,2.323493e+07,8.571212e+06,5.499341e+07
1,강동구,가평우체국,10,4,12,1,108,2017-12-01,-2.2,-7.6,...,2.633287e+06,3.013097e+05,1.348318e+08,1.152319e+08,1.178638e+08,1.311682e+08,5.808194e+07,1.211943e+07,3.380772e+07,1.215478e+07
2,금천구,가평우체국,8,4,12,1,108,2017-12-01,-2.2,-7.6,...,4.146165e+06,4.704613e+05,6.747034e+07,5.679583e+07,6.619319e+07,5.873484e+07,3.965979e+07,1.411274e+07,1.221531e+07,1.333174e+07
3,도봉구,가평우체국,5,4,12,1,108,2017-12-01,-2.2,-7.6,...,7.236443e+05,1.262681e+05,8.305274e+07,6.830370e+07,7.031577e+07,8.020497e+07,3.771521e+07,5.373869e+06,2.106083e+07,1.128051e+07
4,양천구,가평우체국,11,4,12,1,108,2017-12-01,-2.2,-7.6,...,1.439997e+06,2.677776e+05,1.111654e+08,9.373096e+07,9.611992e+07,1.077662e+08,4.986090e+07,7.953973e+06,2.546467e+07,1.644226e+07


In [32]:
df9 = pd.read_excel('input/자치구별경제활동.xls')
df9.head()

,자치구,산출액(①),중간소비(②),산출액(2010년 기준년가격),지역내총부가가치(2010년 기준년가격)
0,종로구,48763786,23159310,47420036,24906839
1,중구,82075885,39013184,82627029,44654355
2,용산구,16730463,7566215,15436810,8205151
3,성동구,20367564,11611307,18826117,7860790
4,광진구,10621481,5479246,9758962,4604213


In [33]:
merge5 = pd.merge(merge4, df9, left_on='도착구', right_on='자치구', how='left')
print(merge5.shape)
merge5.head()

(1338615, 57)


,도착구,발송우체국,수량,요일,월,일,지점,일시,평균기온(°C),최저기온(°C),...,야간인구수(19~08),일최대이동인구수,서울외유입인구수,동일자치구행정동간이동인구수,자치구간이동인구수,자치구,산출액(①),중간소비(②),산출액(2010년 기준년가격),지역내총부가가치(2010년 기준년가격)
0,중구,가평우체국,11,4,12,1,108,2017-12-01,-2.2,-7.6,...,7.566488e+07,8.679954e+07,2.323493e+07,8.571212e+06,5.499341e+07,중구,82075885,39013184,82627029,44654355
1,강동구,가평우체국,10,4,12,1,108,2017-12-01,-2.2,-7.6,...,1.311682e+08,5.808194e+07,1.211943e+07,3.380772e+07,1.215478e+07,강동구,12805305,6428947,11749792,5818433
2,금천구,가평우체국,8,4,12,1,108,2017-12-01,-2.2,-7.6,...,5.873484e+07,3.965979e+07,1.411274e+07,1.221531e+07,1.333174e+07,금천구,29276036,16692938,27483089,11163177
3,도봉구,가평우체국,5,4,12,1,108,2017-12-01,-2.2,-7.6,...,8.020497e+07,3.771521e+07,5.373869e+06,2.106083e+07,1.128051e+07,도봉구,4860561,2419292,4403410,2153533
4,양천구,가평우체국,11,4,12,1,108,2017-12-01,-2.2,-7.6,...,1.077662e+08,4.986090e+07,7.953973e+06,2.546467e+07,1.644226e+07,양천구,10634349,5335045,9771746,4762779


In [34]:
df10 = pd.read_csv('input/구별1인가구수.csv')
df10.head()

,구명,1인가구수
0,종로구,23638
1,중구,20164
2,용산구,33076
3,성동구,39067
4,광진구,54866


In [86]:
merge6 = pd.merge(merge5, df10, left_on='도착구', right_on='구명', how='left')
print(merge6.shape)
merge6.head()

(1338615, 59)


,도착구,발송우체국,수량,요일,월,일,지점,일시,평균기온(°C),최저기온(°C),...,서울외유입인구수,동일자치구행정동간이동인구수,자치구간이동인구수,자치구,산출액(①),중간소비(②),산출액(2010년 기준년가격),지역내총부가가치(2010년 기준년가격),구명_y,1인가구수
0,중구,가평우체국,11,4,12,1,108,2017-12-01,-2.2,-7.6,...,2.323493e+07,8.571212e+06,5.499341e+07,중구,82075885,39013184,82627029,44654355,중구,20164
1,강동구,가평우체국,10,4,12,1,108,2017-12-01,-2.2,-7.6,...,1.211943e+07,3.380772e+07,1.215478e+07,강동구,12805305,6428947,11749792,5818433,강동구,41050
2,금천구,가평우체국,8,4,12,1,108,2017-12-01,-2.2,-7.6,...,1.411274e+07,1.221531e+07,1.333174e+07,금천구,29276036,16692938,27483089,11163177,금천구,32928
3,도봉구,가평우체국,5,4,12,1,108,2017-12-01,-2.2,-7.6,...,5.373869e+06,2.106083e+07,1.128051e+07,도봉구,4860561,2419292,4403410,2153533,도봉구,29469
4,양천구,가평우체국,11,4,12,1,108,2017-12-01,-2.2,-7.6,...,7.953973e+06,2.546467e+07,1.644226e+07,양천구,10634349,5335045,9771746,4762779,양천구,33088


In [87]:
df8.columns.tolist()

Index(['기준일ID', '시군구코드', '총생활인구수', '내국인생활인구수', '장기체류외국인인구수', '단기체류외국인인구수',
       '일최대인구수', '일최소인구수', '주간인구수(09~18)', '야간인구수(19~08)', '일최대이동인구수',
       '서울외유입인구수', '동일자치구행정동간이동인구수', '자치구간이동인구수'],
      dtype='object')

In [88]:
merge6 = merge6.drop(['지역', '구명_x', '기준일ID', '시군구코드', '자치구', '구명_y', '지점', '발송우체국'] + df8.columns.tolist(), axis=1)

In [89]:
merge6.fillna(0, inplace=True)

In [30]:
reverse_seoul = {}
for i, d in seoul.iterrows():
    reverse_seoul[d[0]] = i

reverse_seoul

{'종로구': 0,
 '중구': 1,
 '용산구': 2,
 '성동구': 3,
 '광진구': 4,
 '동대문구': 5,
 '중랑구': 6,
 '성북구': 7,
 '강북구': 8,
 '도봉구': 9,
 '노원구': 10,
 '은평구': 11,
 '서대문구': 12,
 '마포구': 13,
 '양천구': 14,
 '강서구': 15,
 '구로구': 16,
 '금천구': 17,
 '영등포구': 18,
 '동작구': 19,
 '관악구': 20,
 '서초구': 21,
 '강남구': 22,
 '송파구': 23,
 '강동구': 24}

In [91]:
merge6['도착구'].replace(reverse_seoul, inplace=True)

In [92]:
merge6.set_index('일시', inplace=True)

In [93]:
X_test1 = merge6[merge6.index.isin(pd.date_range('2017-12-24', '2017-12-26'))].drop(['수량'], axis=1)
X_test2 = merge6[merge6.index.isin(pd.date_range('2018-08-14', '2018-08-16'))].drop(['수량'], axis=1)
X_test3 = merge6[merge6.index.isin(pd.date_range('2018-09-21', '2018-09-27'))].drop(['수량'], axis=1)
X_test4 = merge6[merge6.index.isin(pd.date_range('2018-10-02', '2018-10-04'))].drop(['수량'], axis=1)
X_test5 = merge6[merge6.index.isin(pd.date_range('2018-10-08', '2018-10-10'))].drop(['수량'], axis=1)

In [94]:
X_train1 = merge6[merge6.index.isin(pd.date_range('2017-12-01', '2017-12-23'))].drop(['수량'], axis=1)
y_train1 = merge6[merge6.index.isin(pd.date_range('2017-12-01', '2017-12-23'))]['수량']

In [95]:
X_train2 = merge6[(merge6.index.isin(pd.date_range('2017-12-01', '2018-08-13'))) & 
                  (~merge6.index.isin(pd.date_range('2017-12-24', '2017-12-26')))].drop(['수량'], axis=1)
y_train2 = merge6[(merge6.index.isin(pd.date_range('2017-12-01', '2018-08-13'))) & 
                  (~merge6.index.isin(pd.date_range('2017-12-24', '2017-12-26')))]['수량']

In [96]:
X_test3 = merge6[(merge6.index.isin(pd.date_range('2017-12-01', '2018-09-20'))) & 
                 (~merge6.index.isin(pd.date_range('2017-12-24', '2017-12-26'))) & 
                 (~merge6.index.isin(pd.date_range('2018-08-14', '2018-08-16')))].drop(['수량'], axis=1)
y_train3 = merge6[(merge6.index.isin(pd.date_range('2017-12-01', '2018-09-20'))) & 
                  (~merge6.index.isin(pd.date_range('2017-12-24', '2017-12-26'))) & 
                  (~merge6.index.isin(pd.date_range('2018-08-14', '2018-08-16')))]['수량']

In [97]:
X_test4 = merge6[(merge6.index.isin(pd.date_range('2017-12-01', '2018-10-02'))) & 
                 (~merge6.index.isin(pd.date_range('2017-12-24', '2017-12-26'))) & 
                 (~merge6.index.isin(pd.date_range('2018-08-14', '2018-08-16'))) &
                 (~merge6.index.isin(pd.date_range('2018-09-21', '2018-09-27')))].drop(['수량'], axis=1)
y_train4 = merge6[(merge6.index.isin(pd.date_range('2017-12-01', '2018-10-02'))) & 
                  (~merge6.index.isin(pd.date_range('2017-12-24', '2017-12-26'))) & 
                  (~merge6.index.isin(pd.date_range('2018-08-14', '2018-08-16'))) &
                  (~merge6.index.isin(pd.date_range('2018-09-21', '2018-09-27')))]['수량']

In [98]:
X_test5 = merge6[(merge6.index.isin(pd.date_range('2017-12-01', '2018-10-07'))) & 
                 (~merge6.index.isin(pd.date_range('2017-12-24', '2017-12-26'))) & 
                 (~merge6.index.isin(pd.date_range('2018-08-14', '2018-08-16'))) &
                 (~merge6.index.isin(pd.date_range('2018-09-21', '2018-09-27'))) &
                 (~merge6.index.isin(pd.date_range('2018-10-02', '2018-10-04')))].drop(['수량'], axis=1)
y_train5 = merge6[(merge6.index.isin(pd.date_range('2017-12-01', '2018-10-07'))) & 
                  (~merge6.index.isin(pd.date_range('2017-12-24', '2017-12-26'))) & 
                  (~merge6.index.isin(pd.date_range('2018-08-14', '2018-08-16'))) &
                  (~merge6.index.isin(pd.date_range('2018-09-21', '2018-09-27'))) &
                  (~merge6.index.isin(pd.date_range('2018-10-02', '2018-10-04')))]['수량']

In [24]:
from sklearn.ensemble import RandomForestRegressor

In [100]:
reg1 = RandomForestRegressor(random_state=42).fit(X_train1, y_train1)

In [101]:
reg1.score(X_train1, y_train1)

0.01605360098025177

In [103]:
X_train1.shape

(91064, 37)

In [45]:
df_test = df1.groupby(['도착구', pd.Grouper(freq='D')])[['수량']].agg(np.sum)

In [46]:
df_test['요일'] = df_test.index.get_level_values(1).dayofweek
df_test['월'] = df_test.index.get_level_values(1).month
df_test['일'] = df_test.index.get_level_values(1).day
df_test.head()

수량  요일   월  일
도착구 발신일자                       
0   2017-12-01  5829   4  12  1
    2017-12-02   514   5  12  2
    2017-12-03   186   6  12  3
    2017-12-04  8525   0  12  4
    2017-12-05  6743   1  12  5

In [47]:
df_test = df_test.unstack(0).stack().reset_index('도착구')

In [41]:
# test_X1 = df_test[df_test.index.isin(pd.date_range('2017-12-01', '2017-12-23'))].drop(['수량'], axis=1) # 발송우체국

# # test_X1.replace(reverse_seoul, inplace=True)
# test_X1 = pd.get_dummies(test_X1, columns=['요일'])
# test_y1 = df_test[df_test.index.isin(pd.date_range('2017-12-01', '2017-12-23'))]['수량']

In [50]:
df_test = df_test[df_test['도착구'] == 0]

In [51]:
test_X1 = df_test[df_test.index.isin(pd.date_range('2017-12-01', '2017-12-23'))].drop(['수량'], axis=1) # 발송우체국

# test_X1.replace(reverse_seoul, inplace=True)
test_X1 = pd.get_dummies(test_X1, columns=['요일'])
test_y1 = df_test[df_test.index.isin(pd.date_range('2017-12-01', '2017-12-23'))]['수량']

In [52]:
reg_test = RandomForestRegressor().fit(test_X1, test_y1)
reg_test.score(test_X1, test_y1)

0.9974635156884502

In [56]:
pd.date_range('2017-12-24', '2017-12-26').tolist() * 25

[Timestamp('2017-12-24 00:00:00', freq='D'),
 Timestamp('2017-12-25 00:00:00', freq='D'),
 Timestamp('2017-12-26 00:00:00', freq='D'),
 Timestamp('2017-12-24 00:00:00', freq='D'),
 Timestamp('2017-12-25 00:00:00', freq='D'),
 Timestamp('2017-12-26 00:00:00', freq='D'),
 Timestamp('2017-12-24 00:00:00', freq='D'),
 Timestamp('2017-12-25 00:00:00', freq='D'),
 Timestamp('2017-12-26 00:00:00', freq='D'),
 Timestamp('2017-12-24 00:00:00', freq='D'),
 Timestamp('2017-12-25 00:00:00', freq='D'),
 Timestamp('2017-12-26 00:00:00', freq='D'),
 Timestamp('2017-12-24 00:00:00', freq='D'),
 Timestamp('2017-12-25 00:00:00', freq='D'),
 Timestamp('2017-12-26 00:00:00', freq='D'),
 Timestamp('2017-12-24 00:00:00', freq='D'),
 Timestamp('2017-12-25 00:00:00', freq='D'),
 Timestamp('2017-12-26 00:00:00', freq='D'),
 Timestamp('2017-12-24 00:00:00', freq='D'),
 Timestamp('2017-12-25 00:00:00', freq='D'),
 Timestamp('2017-12-26 00:00:00', freq='D'),
 Timestamp('2017-12-24 00:00:00', freq='D'),
 Timestamp

In [104]:
test_X1 = df1
test_y1 = df1

,도착구,발송우체국,수량,요일,월,일
발신일자,,,,,,
2017-12-01,중구,가평우체국,11,4,12,1
2017-12-01,강동구,가평우체국,10,4,12,1
2017-12-01,금천구,가평우체국,8,4,12,1
2017-12-01,도봉구,가평우체국,5,4,12,1
2017-12-01,양천구,가평우체국,11,4,12,1
2017-12-01,은평구,가평우체국,11,4,12,1
2017-12-01,종로구,가평우체국,11,4,12,1
2017-12-01,영등포구,가평우체국,9,4,12,1
2017-12-01,강남구,가평우체국,14,4,12,1


In [68]:
X_train1.isnull().sum()

도착구                          0
요일                           0
월                            0
일                            0
평균기온(°C)                     0
최저기온(°C)                     0
최고기온(°C)                     0
일강수량(mm)                 52932
평균 풍속(m/s)               10856
평균 상대습도(%)                   0
전체인구                         0
전체남자                         0
전체여자                         0
65세이상합계                      0
65세이상남자                      0
65세이상여자                      0
사업체수                         0
총종사자수                        0
총여성종사자                       0
총남성종사자                       0
자영업소계                        0
자영업여성종사자                     0
자영업남성종사자                     0
상용근로자소계                      0
상용여성종사자                      0
상용남성종사자                      0
임시직소계                        0
임시직여성종사자                     0
임시직남성종사자                     0
기타소계                         0
기타여성종사자                      0
기타남성종사자                      0
총생활인구수  

In [ ]:
X_test4 = merge6[merge6.index.isin(pd.date_range('2018-10-02', '2018-10-04'))].drop(['수량'], axis=1)
X_test5 = merge6[merge6.index.isin(pd.date_range('2018-10-08', '2018-10-10'))].drop(['수량'], axis=1)


X_train2 = merge6[merge6['일시'].isin(pd.date_range('2017-12-01', '2017-12-23'))].drop(['수량'])
X_train4 = merge6[merge6['일시'].isin(pd.date_range('2017-12-01', '2017-12-23'))].drop(['수량'])
X_train5 = merge6[merge6['일시'].isin(pd.date_range('2017-12-01', '2017-12-23'))].drop(['수량'])
X_train6 = merge6[merge6['일시'].isin(pd.date_range('2017-12-01', '2017-12-23'))].drop(['수량'])

In [54]:
merge6[merge6.index.isin(pd.date_range('2017-12-01', '2017-12-23'))]

,도착구,수량,요일,월,일,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),평균 풍속(m/s),...,야간인구수(19~08),일최대이동인구수,서울외유입인구수,동일자치구행정동간이동인구수,자치구간이동인구수,산출액(①),중간소비(②),산출액(2010년 기준년가격),지역내총부가가치(2010년 기준년가격),1인가구수
일시,,,,,,,,,,,,,,,,,,,,,
2017-12-27,1,8,2,12,27,-6.9,-10.3,-2.6,NaN,1.9,...,7.566488e+07,8.679954e+07,2.323493e+07,8.571212e+06,5.499341e+07,82075885,39013184,82627029,44654355,20164
2017-12-27,20,7,2,12,27,-6.9,-10.3,-2.6,NaN,1.9,...,1.429741e+08,5.823499e+07,9.116628e+06,3.258344e+07,1.653492e+07,9583809,4749349,8812965,4356617,106865
2017-12-27,21,18,2,12,27,-6.9,-10.3,-2.6,NaN,1.9,...,1.437142e+08,1.132019e+08,3.664846e+07,2.312499e+07,5.342847e+07,50765577,24398811,47302449,24111257,39702
2017-12-27,18,9,2,12,27,-6.9,-10.3,-2.6,NaN,1.9,...,1.253557e+08,9.165104e+07,2.458251e+07,2.355239e+07,4.351615e+07,47668736,23466433,45828278,23058184,51151
2017-12-27,6,5,2,12,27,-6.9,-10.3,-2.6,NaN,1.9,...,9.979067e+07,4.334684e+07,6.861309e+06,2.600694e+07,1.047860e+07,7236054,3795072,6580746,3063836,47543
2017-12-27,22,18,2,12,27,-6.9,-10.3,-2.6,NaN,1.9,...,1.896922e+08,1.591061e+08,4.928203e+07,3.210793e+07,7.771616e+07,105791881,53056324,99047720,49155046,62774
2017-12-27,24,5,2,12,27,-6.9,-10.3,-2.6,NaN,1.9,...,1.311682e+08,5.808194e+07,1.211943e+07,3.380772e+07,1.215478e+07,12805305,6428947,11749792,5818433,41050
2017-12-27,8,4,2,12,27,-6.9,-10.3,-2.6,NaN,1.9,...,8.426276e+07,3.516245e+07,3.446473e+06,1.953363e+07,1.218235e+07,4871189,2468595,4447549,2131237,39082
2017-12-27,15,11,2,12,27,-6.9,-10.3,-2.6,NaN,1.9,...,1.490520e+08,6.780220e+07,1.608484e+07,3.493689e+07,1.678047e+07,18452314,9617588,17381843,7950651,68237


In [ ]:
df.tail()

In [ ]:
test1 = df1[df1.index.isin(pd.date_range('2017-12-01', '2017-12-23'))].copy()

In [ ]:
test1['발송우체국'] = test1['발송우체국'].astype('category')

In [ ]:
test1['발송우체국'] = test1['발송우체국'].cat.codes

In [ ]:
ddweather = 

In [ ]:
test1 = pd.get_dummies(test1, columns=['요일'])

In [ ]:
df1.pivot_table(index=['도착구', '발송우체국'], values='수량', aggfunc='count').fillna(0)

In [ ]:
df1.groupby(by='도착구')['발송우체국'].agg('count').sort_values()

In [ ]:
test1.loc[:,['발송우체국']]

In [ ]:
df1[df1.index.isin(pd.date_range('2017-12-01', '2017-12-23'))]

In [ ]:
df1.plot(kind='barh', x='요일', y='수량')
plt.show()

In [ ]:
grouped_posts = df1.groupby(by=['발송우체국', '도착구', pd.Grouper(freq='D')])[['수량']].agg(np.sum)

In [ ]:
df.shape

In [ ]:
grouped_posts.unstack(0).stack().reset_index()

In [ ]:
print(len(grouped_posts.index.get_level_values(0).unique()))
grouped_posts.index.get_level_values(0).unique()[:100]

In [ ]:
idx = pd.IndexSlice
subset_posts = grouped_posts.loc[idx['가평우체국']]


In [ ]:
grouped_posts

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.pointplot(data=subset_posts, x=subset_posts.index.get_level_values(1), 
              y='수량', hue=subset_posts.index.get_level_values(0))
plt.show()

In [ ]:
df2 = df.copy()
df2.head()

In [ ]:
df2['요일'] = df2['발신일자'].dt.dayofweek
df2['월'] = df2['발신일자'].dt.month
df2['일'] = df2['발신일자'].dt.day
df2.head()

In [ ]:
df2.sort_values(by='발신일자', ascending=True)[:5]

In [ ]:
df3 = df2.copy()
df3['공휴일'] = None

In [ ]:
df3.loc[df3['발신일자'] == '2017-12-20', '공휴일'] = '대통령선거'
df3.loc[df3['발신일자'] == '2017-12-24', '공휴일'] = '크리스마스 이브'
df3.loc[df3['발신일자'] == '2017-12-25', '공휴일'] = '크리스마스'

In [ ]:
df3[df3['발신일자'].isin(['2017-12-24', '2017-12-25', '2017-12-26'])] # 크리스마스 연휴

In [ ]:
df3[df3['발신일자'].isin(['2018-08-14', '2018-08-15', '2018-08-16'])] # 광복절

In [ ]:
df3[df3['발신일자'].isin(pd.date_range('2018-09-21', '2018-09-27'))] # 추석

In [ ]:
df3[df3['발신일자'].isin(pd.date_range('2018-10-02', '2018-10-04'))] # 개천절

In [ ]:
df3[df3['발신일자'].isin(pd.date_range('2018-10-08', '2018-10-10'))] # 한글날

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
test1 = df3[df3['발신일자'].isin(pd.date_range('2017-12-01', '2017-12-23'))]
test1.shape

In [ ]:
test1.set_index('발신일자', inplace=True)

In [ ]:
test1.drop(['발송우체국', '도착구', '요일', '월', '일', '공휴일'], axis=1, inplace=True)
test1.head()

In [ ]:
test2 = test1.groupby(pd.Grouper(freq='D'))[['수량']].agg(np.sum)
test2.plot(kind='line')
plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(15, 7))
# plot_acf(test1, ax=ax)
# plt.show()
fig, ax = plt.subplots(figsize=(15, 7))
plot_acf(test2, ax=ax)
plt.show()

In [ ]:
# fig, ax = plt.subplots(figsize=(15, 7))
# plot_pacf(test1, ax=ax)
# plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
diff1 = test2.diff(periods=1)
diff1.plot(ax=ax)
plot_acf(diff1)
plt.show()

In [ ]:
model = ARIMA(test2, order=(0,0,1))
model_fit = model.fit(trend='c', full_output=True, disp=1, maxiter=3000, )
print(model_fit.summary())

fig, ax = plt.subplots(figsize=(12, 8))
model_fit.plot_predict(ax=ax)

In [ ]:
model_fit.forecast(steps=3)[0]

In [ ]:
test1 = df3[(df3['발신일자'].isin(pd.date_range('2017-12-01', '2017-12-23'))) & (df['도착구'] == 1)]
test1 = test1.set_index('발신일자')

In [ ]:
test1 = test1.drop(['발송우체국', '도착구', '요일', '월', '일', '공휴일'], axis=1)
test1.head()

In [ ]:
test1 = test1.groupby(pd.Grouper(freq='D'))[['수량']].agg(np.sum)
test1.head()

In [ ]:
model = ARIMA(test1, order=(0,0,1))
model_fit = model.fit(trend='c', full_output=True, disp=1, maxiter=5000)
print(model_fit.summary())

fig, ax = plt.subplots(figsize=(12, 8))
model_fit.plot_predict(ax=ax)

In [ ]:
test1 = df3[(df3['발신일자'].isin(pd.date_range('2017-12-01', '2017-12-23'))) & (df['도착구'] == 1)]
test1 = test1.set_index('발신일자')
test1.head()

In [ ]:
test1 = test1.groupby(pd.Grouper(freq='D'))[['수량']].agg(np.sum)
test1.plot(kind='line')
plt.show()

In [ ]:
test1['요일'] = test1.index.dayofweek
test1['월'] = test1.index.month
test1['일'] = test1.index.day
test1.head()

In [ ]:
test1['발송우체국'] = test1['발송우체국'].astype('category')

In [ ]:
test1['발송우체국'] = test1['발송우체국'].cat.codes

In [ ]:
test1 = pd.get_dummies(test1, columns=['요일'])

In [ ]:
test1['수량'].plot(kind='line')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
X = test1.drop(['수량'], axis=1)
y = test1['수량']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
reg = RandomForestRegressor().fit(X_train, y_train)

In [ ]:
reg.score(X_train, y_train)

In [ ]:
reg.score(X_test, y_test)

In [ ]:
pred1 = reg.predict(X_test)
pred2 = reg.predict(X_train)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
plt.plot(list(range(len(X_train))), y_train)
plt.plot(list(range(len(X_train))), pred2)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
plt.plot(list(range(len(pred))), pred)
plt.plot(list(range(len(pred))), y_test)
plt.show()

In [ ]:
range(len(pred))

In [ ]:
len(pred)

In [ ]:
df3[df3['발신일자'] == '2017-12-20']

In [ ]:
df3[df3['발신일자'] == '2017-12-27']

In [ ]:
df3[df3['발신일자'] == '2017-12-20'] 

In [ ]:
df3[df3['발신일자'] == '2017-12-25']

In [ ]:
a.groupby(by=['post', pd.Grouper(freq='D')])[['num']].agg(np.mean)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def find_outliers(estimator, X, y, sigma=3, print_outilers=False, savefig=False):
    """
    Find outliers to use residulas
    
    Parameters
    __________
    
    input : estimator, X, y, sigma
    
    estimator : scikit-learn
    """
    estimator.fit(X, y)
    y_pred = pd.Series(estimator.predict(X), index=y.index)
    
    # Calculate residuals between the model prediction and true y values
    resid = y - y_pred
    resid_mean = np.mean(resid)
    resid_std = np.std(resid)

    # Calculate z statistic, define outliers to be where |z|>sigma
    z = (resid - resid_mean) / resid_std    
    outliers = z[abs(z) > sigma].index
    
    # Print and plot the results
    print('R2 = {}'.format(estimator.score(X,y)))
    print('RMSE = {}'.format(rmse(y, y_pred)))
    print('---------------------------------------')

    print('mean of residuals: {}'.format(resid_mean))
    print('std of residuals: {}'.format(resid_std))
    print('---------------------------------------')

    print('\noutliers: {}'.format(len(outliers)))
    
    if print_outilers and len(outliers) < 100:
        print(outliers.tolist())

    fig, axes = plt.subplots(1, 3, figsize=(15,5))
    
    axes.flat[0].plot(y, y_pred, 'b.')
    axes.flat[0].plot(y.loc[outliers], y_pred.loc[outliers], 'rx')
    axes.flat[0].legend(['Normal','Outlier'])
    axes.flat[0].set_xlabel('SalePrice')
    axes.flat[0].set_ylabel('Prediction')
    axes.flat[0].tick_params(axis='x', rotation=-45)

    axes.flat[1].plot(y, y - y_pred, 'b.')
    axes.flat[1].plot(y.loc[outliers], y.loc[outliers] - y_pred.loc[outliers], 'rx')
    axes.flat[1].legend(['Normal','Outlier'])
    axes.flat[1].set_xlabel('SalePrice')
    axes.flat[1].set_ylabel('Residuals')
    axes.flat[1].tick_params(axis='x', rotation=-45)

    z.plot.hist(bins=60, ax=axes.flat[2], color='blue')
    z.loc[outliers].plot.hist(color='r', bins=50, ax=axes.flat[2])
    axes.flat[1].legend(['Normal','Outlier'])
    axes.flat[1].set_xlabel('z-score')
    axes.flat[1].tick_params(axis='x', rotation=-45)
    fig.tight_layout()
    plt.show()
    
    if savefig:
        fig.savefig('outliers.png')
    
    return outliers

In [ ]:
import numpy as np


def rmse(y_true, y_pred):
    """
    Root mean squared error
    
    Parameters
    ----------
    
    input : numpy array
    """
    diff = y_pred - y_true
    sum_sq = np.sum(diff**2)    
    n = len(y_pred)
    return np.sqrt(sum_sq / n)

In [ ]:
b2 = b1.reset_index()

In [ ]:
b2.head()

In [ ]:
b3 = b2.drop(['date'], axis=1)

In [ ]:
b3['post'] = b3['post'].astype('category')

In [ ]:
b3['post'] = b3['post'].cat.codes

In [ ]:
b4 = b2.copy()

In [ ]:
b4.set_index('date', inplace=True)
b4.drop(['post'], axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(b3, test_size=0.2, random_state=42)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
order = (2, 1, 2)
model = ARIMA(b4, order)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
plot_acf(b4, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
plot_pacf(b4, ax=ax)
plt.show()

In [ ]:
diff_1 = b4.diff(periods=1)
diff_1.plot()
plot_acf(diff_1)
plt.show()

In [ ]:
diff_1 = b4.diff(periods=1)
diff_1.plot()
plot_pacf(diff_1)
plt.show()

In [ ]:
model = ARIMA(b4, order=(0,1,1))
model_fit = model.fit(trend='c', full_output=True, disp=1)
print(model_fit.summary())

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
model_fit.plot_predict(ax=ax)

In [ ]:
grouped_weather = copied_weather.groupby(by=['지점', pd.Grouper(freq='D')])[['기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)']].agg(['min', 'max', 'mean']).fillna(0)